In [ ]:
import requests
import pandas as pd
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc

rc('font', family='AppleGothic')

plt.rcParams['axes.unicode_minus'] = False

# api url
base_url = 'http://apis.data.go.kr/1160100/service/GetStockSecuritiesInfoService/getStockPriceInfo?serviceKey=9oUviJPMmodfrNLBPYD0czH%2BOoPnArWcnHsw6T5g2TwnKEs1dwXCp%2FI%2BNfeEGngIAK74J550ZjxJ0CxJy4cK7Q%3D%3D&resultType=json'

# rsi 함수
def calc_momentum(url):
    response = requests.get(url)
    
    if response.status_code == 200:
        contents = response.text  # JSON 데이터 가져오기
    else:
        print(f"API 호출 실패: {response.status_code}")
    
    json_ob = json.loads(contents)

    data = json_ob['response']['body']['items']['item']
    selected_data = []

    for item in data:
        selected_item = {
            'basDt': item['basDt'],
            'itmsNm': item['itmsNm'],
            'mrktCtg': item['mrktCtg'],
            'clpr': item['clpr'],
            'vs': item['vs'],
            'fltRt': item['fltRt'],
            'mkp': item['mkp'],
            'hipr': item['hipr'],
            'lopr': item['lopr']
        }
        selected_data.append(selected_item)
        
    dataframe = pd.DataFrame(selected_data)

    # 데이터 프레임 열 이름 변경
    dataframe = dataframe.rename(columns={
        'basDt': '날짜',
        'itmsNm': '종목명',
        'mrktCtg': '시장구분',
        'clpr': '종가',
        'vs': '거래량',
        'fltRt': '등락률',
        'mkp': '시가총액',
        'hipr': '고가',
        'lopr': '저가'
    })
    
    # 날짜를 날짜 형식으로 변환
    dataframe['날짜'] = pd.to_datetime(dataframe['날짜'], format='%Y%m%d')

    # 종가를 수치로 변환
    dataframe['종가'] = pd.to_numeric(dataframe['종가'])

    # 날짜를 인덱스로 설정
    dataframe.set_index('날짜', inplace=True)
    
    # 모멘텀 계산 (2일 간격)
    period = 2
    dataframe['모멘텀'] = dataframe['종가'].diff(period)

    # 모멘텀 시각화
    plt.figure(figsize=(10, 5))
    plt.plot(dataframe.index, dataframe['모멘텀'], marker='o', linestyle='-', color='g')
    #plt.axhline(y=0, color='r', linestyle='--', label='Zero Line')
    plt.title('Momentum Over Time')
    plt.xticks(rotation=45)  # x축 레이블에 45도 각도 적용
    plt.xlabel('Date')
    plt.ylabel('Momentum')
    plt.legend()
    return plt.show()
    
